In [1]:
import os
from pathlib import Path
import comtypes.client
import tkinter
import pyautogui as pag
import time
from utils.helper import connect_to_sap2000, get_node_distance, get_lp, click_image, click_image_offset, maximize_window_temporarily, make_icon_finder
from utils.excel_helpers import open_excel, save_excel
pag.FAILSAFE = True
pag.PAUSE = 1
time.sleep(1)

In [2]:
ICONS = Path("icons-unr")
icon = make_icon_finder(ICONS)
icon("define-button.PNG")

'icons-unr\\define-button.PNG'

In [3]:
sap_model = connect_to_sap2000()

if sap_model.GetModelIsLocked():
    sap_model.SetModelIsLocked(False)
    print("✓ Model unlocked")

ret = sap_model.SelectObj.All()
if not ret: print("✓ Column selected")


✓ Connected to SAP2000
✓ Column selected


In [4]:
sec_name = sap_model.PropFrame.GetNameList()[1][0]
I_gross = sap_model.PropFrame.GetSectProps(sec_name)[5]
print("MOI:", I_gross)

MOI: 10645.472607960633


In [5]:
L = get_node_distance(sap_model, 1, 2)
db = sap_model.PropFrame.SDShape.GetSolidCircle(sec_name, "Circle1")[10]
if db in ['None', None, '']: print("Input db manually") 
fye = sap_model.PropMaterial.GetORebar_1("Long Rebar")[2]

Input db manually


In [6]:
try:
    axial_load = sap_model.PointObj.GetLoadForce("2")[7][1]
except:
    axial_load = 0
print("Axial Load:", axial_load)

Axial Load: -117.0


In [7]:
db = 1
hinge_length = get_lp(L, db, fye)
print("Hinge Length:", hinge_length)

Hinge Length: 20.4


In [9]:
click_image(icon("define-button.PNG"), 2)
click_image(icon("section-properties.png"))
click_image(icon("frame-sections.png"))
click_image(icon("modify-property.PNG"))
click_image(icon("section-designer.PNG"))
time.sleep(3.5)
click_image(icon("display-SD.PNG"))
click_image(icon("show-m-phi.png"))

✗ Error: 
✗ Error: 
✗ Error: 
✗ Error: 
✗ Error: 
✗ Error: 
✗ Error: 


False

In [16]:
click_image_offset(icon("idealized.PNG"), x_offset_ratio=0.1)
click_image_offset(icon("axial.PNG"), x_offset_ratio=0.8)
pag.press('backspace', presses=5)
pag.write(f"{axial_load}")
click_image_offset(icon("details-m-phi.PNG"))
maximize_window_temporarily()
time.sleep(0.5)
click_image_offset(icon("I_crack.PNG"), x_offset_ratio=1)
pag.drag(80, duration=0.5)
pag.hotkey('ctrl', 'c')
I_crack = float(tkinter.Tk().clipboard_get())
print("I_crack=", I_crack )
click_image_offset(icon("ok.PNG"))
click_image_offset(icon("done.PNG"))
time.sleep(1)
click_image_offset(icon("done2.PNG"))
pag.press('esc')
pag.press('esc')

✓ Clicked icons\idealized.PNG at (1093, 668) [x_ratio=0.1, y_ratio=0.5]
✓ Clicked icons\axial.PNG at (1277, 704) [x_ratio=0.8, y_ratio=0.5]
✓ Clicked icons\details-m-phi.PNG at (1668, 856) [x_ratio=0.5, y_ratio=0.5]
✓ Clicked icons\I_crack.PNG at (90, 720) [x_ratio=1, y_ratio=0.5]
I_crack= 2115.7059
✓ Clicked icons\ok.PNG at (960, 980) [x_ratio=0.5, y_ratio=0.5]
✓ Clicked icons\done.PNG at (1847, 894) [x_ratio=0.5, y_ratio=0.5]
✓ Clicked icons\done2.PNG at (1563, 920) [x_ratio=0.5, y_ratio=0.5]


In [17]:
k_mod = round(I_crack / I_gross, 5)
sap_model.PropFrame.SetModifiers(sec_name, [1, 1, 1, 1, k_mod, k_mod, 1, 1])

[(1.0, 1.0, 1.0, 1.0, 0.36819, 0.36819, 1.0, 1.0), 0]

In [ ]:
ret = sap_model.SelectObj.All()
click_image_offset(icon("assign.PNG"), n=1)
click_image_offset(icon("assign-frame.PNG"))
click_image_offset(icon("hinges.PNG"))
with pag.hold('ctrl'):
    click_image_offset(icon("modify-hinge.PNG"))
time.sleep(0.5)
click_image_offset(icon("hinge-length.PNG"), x_offset_ratio=1.4)
pag.press('backspace', presses=7)
pag.write(f"{hinge_length}")
time.sleep(0.5)
click_image_offset(icon("ok-hinge-length.PNG"))
click_image_offset(icon("ok-hinge.PNG"))
time.sleep(1.5)

✓ Clicked icons-unr\assign.PNG at (304, 36) [x_ratio=0.5, y_ratio=0.5]
✓ Clicked icons-unr\assign-frame.PNG at (326, 88) [x_ratio=0.5, y_ratio=0.5]
✓ Clicked icons-unr\hinges.PNG at (620, 432) [x_ratio=0.5, y_ratio=0.5]
✓ Clicked icons-unr\modify-hinge.PNG at (1489, 355) [x_ratio=0.5, y_ratio=0.5]
✓ Clicked icons-unr\hinge-length.PNG at (1218, 338) [x_ratio=1.4, y_ratio=0.5]
✓ Clicked icons-unr\ok-hinge-length.PNG at (898, 548) [x_ratio=0.5, y_ratio=0.5]
✓ Clicked icons-unr\ok-hinge.PNG at (1117, 774) [x_ratio=0.5, y_ratio=0.5]


In [158]:
sap_model.Analyze.RunAnalysis()
time.sleep(1)
pag.hotkey('alt')
pag.press('p')
pag.press('C')
# click_image_offset(os.path.join("icons", "display.PNG"), n=2)
# click_image_offset(os.path.join("icons", "pushover.PNG"))
pag.hotkey('shift', 'f12')
pag.hotkey('alt', 'f')
pag.press('t')
pag.press('e')